In [ ]:
# 라이브러리
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
from collections import deque
import pickle
import pyperclip
import json
import pandas as pd
import tkinter as tk
from tkinter import messagebox
import os
import subprocess
import random

# 크롤링 웹드라이버 세팅
def webdriver_set():
    op = webdriver.ChromeOptions()
    op.add_argument("window-size=1000,1000")
    user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
    op.add_argument(f"user-agent={user_agent}")
    chrome_service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service = chrome_service, options=op)
    return driver

# 네이버 로그인 계정 세팅
def naver_login(naverid, naverpw):

    driver.get("https://nid.naver.com/nidlogin.login")

    try: 
        driver.find_element(By.CSS_SELECTOR,'#id')
        time.sleep(random.uniform(2,4))

        pyperclip.copy(id)
        driver.find_element(By.CSS_SELECTOR,'#id').send_keys(Keys.CONTROL+'v') # id 채우기
        time.sleep(random.uniform(1,2))

        pyperclip.copy(pw)
        secure='blank'
        driver.find_element(By.CSS_SELECTOR,'#pw').send_keys(Keys.CONTROL + 'v') # pw 채우기
        time.sleep(random.uniform(1,2))
        pyperclip.copy(secure) 

        driver.find_element(By.XPATH,'//*[@id="log.login"]').click() # 로그인
    except: 
        print("no such element")          #예외처리





# 본인 계정정보 입력
id = ""
pw = ""


# 데이터 저장 관련 설정
file_name = 'results.csv'

# 작업이력이 있으면 그 csv파일을 열어서 가장 마지막에 작업한 글의 URL부터 작업시작, 작업이력 없다면 URL 직접 지정
if not os.path.exists(file_name):
    url_start = "https://cafe.naver.com/imsanbu/70682457" # 크롤링 시작할 첫 URL 주소. 이 글 기준으로 다음글/이전글 옮겨다니며 크롤링 **2023년 마지막 글 https://cafe.naver.com/imsanbu/69101951, 9/10 기준 시작 글 https://cafe.naver.com/imsanbu/72398515

    print('기작업된 데이터 없음. 새 작업 시작')
else:
    df = pd.read_csv(file_name)
    # URL 열에서 가장 마지막 값 가져오기
    url_start = df['URL'].iloc[-1]
    row_count = df.shape[0]
    print(f"기작업된 데이터 수: {row_count}")



In [ ]:
data = [] # data 초기화

target_count = 30000  # 탐색할 목표 게시글 수


# 웹드라이버 실행
driver = webdriver_set()

# 로그인
naver_login(id, pw)


driver.implicitly_wait(60) # 최장 대기시간 
driver.get(url_start) # 첫 URL 열기
driver.switch_to.frame("cafe_main") 
time.sleep(random.uniform(1,2))

for i in tqdm(range(1, target_count+1)): # 목표 게시글 수 만금 반복

    try:
        driver.find_element(By.CSS_SELECTOR, "#app > div > div > div.ArticleTopBtns > div.right_area > a.BaseButton.btn_prev.BaseButton--skinGray.size_default").click() #다음글로 이동. btn_next를 btn_prev로 바꾸면 이전 글로 이동함.
        time.sleep(random.uniform(3,5))
    except:
        break
    
    try:
        url_info = driver.find_element(By.XPATH, '//*[@id="spiButton"]').get_attribute('data-url') # 게시글 url
        num_info = url_info.split("/")[-1] # 게시글 번호
        date_info = driver.find_element(By.CLASS_NAME, 'date').text # 게시글 날짜
        author_info = driver.find_element(By.CLASS_NAME, 'nickname').text # 게시글 작성자
        title_info = driver.find_element(By.CLASS_NAME, 'title_text').text # 게시글 제목
        text_info = driver.find_element(By.CLASS_NAME, 'se-main-container').text # 게시글 내용
    except:
        pass

    # data에 게시글 정보 추가
    if len(text_info) > 0:
        data.append([url_info, num_info, date_info, author_info, title_info, text_info])

    # 2023년 이전 글은 작업하지 않음
    if date_info[:4] <= '2023':
        break

    # 매 10번째 작업마다 data 내용을 데이터프레임에 입력하고 저장
    if i % 10 == 0:
        df_new = pd.DataFrame(data, columns=['URL', '글번호', '날짜', '작성자', '제목', '내용'])
        if not os.path.exists(file_name):
            df_new.to_csv(f"./{file_name}", index=False, encoding='utf-8-sig')  # CSV 파일로 저장
            data = [] # data 초기화
        else:
            df = pd.read_csv(file_name)
            df = pd.concat([df, df_new])
            df.to_csv(f"./{file_name}", index=False, encoding='utf-8-sig')
            data = [] # data 초기화
        

        # 매 100번째 작업마다 웹드라이버 재시작
        if i % 100 == 0:
            df = pd.read_csv(file_name)
            # URL 열에서 가장 마지막 값 가져오기
            url_start = df['URL'].iloc[-1]

            del [[data]]
            del [[df_new]]
            del [[df]]
            data = []


            driver.quit()
            time.sleep(random.uniform(1,1.5))
            driver = webdriver_set()
            time.sleep(random.uniform(1,1.5))
            naver_login(id, pw)
            time.sleep(random.uniform(1,1.5))

            driver.get(url_start)
            driver.switch_to.frame("cafe_main")
            time.sleep(random.uniform(2,3))

# 작업완료시 팝업 출력
def finish_msg():
    driver.quit()
    root = tk.Tk()
    root.withdraw()
    messagebox.showinfo("완료", "작업종료")
    
    root.destroy()

finish_msg()
